# keras 点评文本情感分析

In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('/Users/liming/Downloads/review_star.csv')
print(data.shape)
data.head()

(10000, 3)


,reviewid,reviewbody,star
0,661655779,感谢大众点评的vip会员卷。很多好吃的都打折。去天河城吃饭顺便把喝的换了。两杯茶才9.9。简...,40
1,661662037,（9月26日就餐）这家的服务态度真是很奇葩了：在等餐时看到服务员在端豆浆上桌前，突然在碗里发...,5
2,661662167,除了贵 没毛病,35
3,661674219,亲民，家门口，味道可以，主要是在家附近，吃点家常菜还是比较方便的，不做饭就在这里吃点，做的也...,45
4,661677846,之前在会展中心店按过，觉得很不错，说上梅林也有一家店，刚好在附近办事，所以来了这里，装修风格...,50


In [2]:
# 定义函数：根据用户评的星级来估计sentiment（情感）
def make_label(star):
    if star >=30:
        return 1
    else:
        return 0
# 运用 apply 方法得到新列
data["sentiment"] = data.star.apply(make_label)
data.head()

,reviewid,reviewbody,star,sentiment
0,661655779,感谢大众点评的vip会员卷。很多好吃的都打折。去天河城吃饭顺便把喝的换了。两杯茶才9.9。简...,40,1
1,661662037,（9月26日就餐）这家的服务态度真是很奇葩了：在等餐时看到服务员在端豆浆上桌前，突然在碗里发...,5,0
2,661662167,除了贵 没毛病,35,1
3,661674219,亲民，家门口，味道可以，主要是在家附近，吃点家常菜还是比较方便的，不做饭就在这里吃点，做的也...,45,1
4,661677846,之前在会展中心店按过，觉得很不错，说上梅林也有一家店，刚好在附近办事，所以来了这里，装修风格...,50,1


In [3]:
data["reviewbody"] = data["reviewbody"].astype(str)
data.head()

,reviewid,reviewbody,star,sentiment
0,661655779,感谢大众点评的vip会员卷。很多好吃的都打折。去天河城吃饭顺便把喝的换了。两杯茶才9.9。简...,40,1
1,661662037,（9月26日就餐）这家的服务态度真是很奇葩了：在等餐时看到服务员在端豆浆上桌前，突然在碗里发...,5,0
2,661662167,除了贵 没毛病,35,1
3,661674219,亲民，家门口，味道可以，主要是在家附近，吃点家常菜还是比较方便的，不做饭就在这里吃点，做的也...,45,1
4,661677846,之前在会展中心店按过，觉得很不错，说上梅林也有一家店，刚好在附近办事，所以来了这里，装修风格...,50,1


In [4]:
# 首先对评论数据进行分词
import jieba

def chinese_word_cut(mytext):
 return " ".join(jieba.cut(mytext))

data['cut_comment'] = data["reviewbody"].apply(chinese_word_cut)
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zd/qhg48cw17_ncqf0rl48wz5rh0000gp/T/jieba.cache
Loading model cost 0.545 seconds.
Prefix dict has been built successfully.


,reviewid,reviewbody,star,sentiment,cut_comment
0,661655779,感谢大众点评的vip会员卷。很多好吃的都打折。去天河城吃饭顺便把喝的换了。两杯茶才9.9。简...,40,1,感谢 大众 点评 的 vip 会员 卷 。 很多 好吃 的 都 打折 。 去 天河城 吃饭 ...
1,661662037,（9月26日就餐）这家的服务态度真是很奇葩了：在等餐时看到服务员在端豆浆上桌前，突然在碗里发...,5,0,（ 9 月 26 日 就餐 ） 这家 的 服务态度 真是 很 奇葩 了 ： 在 等 餐时 看...
2,661662167,除了贵 没毛病,35,1,除了 贵 没 毛病
3,661674219,亲民，家门口，味道可以，主要是在家附近，吃点家常菜还是比较方便的，不做饭就在这里吃点，做的也...,45,1,亲民 ， 家门口 ， 味道 可以 ， 主要 是 在家 附近 ， 吃点 家常菜 还是 比较 方...
4,661677846,之前在会展中心店按过，觉得很不错，说上梅林也有一家店，刚好在附近办事，所以来了这里，装修风格...,50,1,之前 在 会展中心 店 按 过 ， 觉得 很 不错 ， 说 上 梅林 也 有 一家 店 ， ...


In [5]:
maxlen = 100 #截断词数
min_count = 5 #出现次数少于该值的词扔掉。这是最简单的降维方法
 
content = []
for i in data['cut_comment']:
	content.extend(i)

abc = pd.Series(content).value_counts()
abc = abc[abc >= min_count]
abc[:] = range(1, len(abc)+1)
abc[''] = 0 #添加空字符串用来补全
word_set = set(abc.index)

def doc2num(s, maxlen):
    s = [i for i in s if i in word_set]
    s = s[:maxlen] + ['']*max(0, maxlen-len(s))
    return list(abc[s])
 
data['doc2num'] = data['cut_comment'].apply(lambda s: doc2num(s, maxlen))
data.head()

,reviewid,reviewbody,star,sentiment,cut_comment,doc2num
0,661655779,感谢大众点评的vip会员卷。很多好吃的都打折。去天河城吃饭顺便把喝的换了。两杯茶才9.9。简...,40,1,感谢 大众 点评 的 vip 会员 卷 。 很多 好吃 的 都 打折 。 去 天河城 吃饭 ...,"[53, 401, 1, 32, 435, 1, 19, 202, 1, 3, 1, 886..."
1,661662037,（9月26日就餐）这家的服务态度真是很奇葩了：在等餐时看到服务员在端豆浆上桌前，突然在碗里发...,5,0,（ 9 月 26 日 就餐 ） 这家 的 服务态度 真是 很 奇葩 了 ： 在 等 餐时 看...,"[470, 1, 403, 1, 495, 1, 199, 327, 1, 254, 1, ..."
2,661662167,除了贵 没毛病,35,1,除了 贵 没 毛病,"[676, 9, 1, 345, 1, 1, 1, 42, 1, 571, 1412, 0,..."
3,661674219,亲民，家门口，味道可以，主要是在家附近，吃点家常菜还是比较方便的，不做饭就在这里吃点，做的也...,45,1,亲民 ， 家门口 ， 味道 可以 ， 主要 是 在家 附近 ， 吃点 家常菜 还是 比较 方...,"[538, 779, 1, 2, 1, 27, 174, 54, 1, 2, 1, 22, ..."
4,661677846,之前在会展中心店按过，觉得很不错，说上梅林也有一家店，刚好在附近办事，所以来了这里，装修风格...,50,1,之前 在 会展中心 店 按 过 ， 觉得 很 不错 ， 说 上 梅林 也 有 一家 店 ， ...,"[142, 113, 1, 28, 1, 56, 914, 77, 70, 1, 29, 1..."


In [6]:
#按keras的输入要求来生成数据
x = np.array(list(data['doc2num']))
y = np.array(list(data['sentiment']))
y = y.reshape((-1,1)) #调整标签形状

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [8]:
#建立模型
model = Sequential()
model.add(Embedding(len(abc), 256, input_length=maxlen))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
batch_size = 128

# 划分测试集和训练集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=22)
 
model.fit(x_train, y_train, batch_size = batch_size, nb_epoch=10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Epoch 1/10
8000/8000 [==============================] - 43s 5ms/step - loss: 0.1966 - accuracy: 0.9515
Epoch 2/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.1863 - accuracy: 0.9515
Epoch 3/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.1593 - accuracy: 0.9557
Epoch 4/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.1396 - accuracy: 0.9609
Epoch 5/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.1268 - accuracy: 0.9668
Epoch 6/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.1204 - accuracy: 0.9685
Epoch 7/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.1139 - accuracy: 0.9714
Epoch 8/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.1345 - accuracy: 0.9639
Epoch 9/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.1535 - accuracy: 0.9516
Epoch 10/10
8000/8000 [==============================] - 45s 6ms/step - l

In [11]:
model.evaluate(x_test, y_test, batch_size = batch_size)

2000/2000 [==============================] - 4s 2ms/step


[0.23422445487976074, 0.940500020980835]

In [15]:
nn_result = model.predict_classes(x)
data['nn_result'] = nn_result
data.head()

,reviewid,reviewbody,star,sentiment,cut_comment,doc2num,nn_result
0,661655779,感谢大众点评的vip会员卷。很多好吃的都打折。去天河城吃饭顺便把喝的换了。两杯茶才9.9。简...,40,1,感谢 大众 点评 的 vip 会员 卷 。 很多 好吃 的 都 打折 。 去 天河城 吃饭 ...,"[53, 401, 1, 32, 435, 1, 19, 202, 1, 3, 1, 886...",1
1,661662037,（9月26日就餐）这家的服务态度真是很奇葩了：在等餐时看到服务员在端豆浆上桌前，突然在碗里发...,5,0,（ 9 月 26 日 就餐 ） 这家 的 服务态度 真是 很 奇葩 了 ： 在 等 餐时 看...,"[470, 1, 403, 1, 495, 1, 199, 327, 1, 254, 1, ...",0
2,661662167,除了贵 没毛病,35,1,除了 贵 没 毛病,"[676, 9, 1, 345, 1, 1, 1, 42, 1, 571, 1412, 0,...",1
3,661674219,亲民，家门口，味道可以，主要是在家附近，吃点家常菜还是比较方便的，不做饭就在这里吃点，做的也...,45,1,亲民 ， 家门口 ， 味道 可以 ， 主要 是 在家 附近 ， 吃点 家常菜 还是 比较 方...,"[538, 779, 1, 2, 1, 27, 174, 54, 1, 2, 1, 22, ...",1
4,661677846,之前在会展中心店按过，觉得很不错，说上梅林也有一家店，刚好在附近办事，所以来了这里，装修风格...,50,1,之前 在 会展中心 店 按 过 ， 觉得 很 不错 ， 说 上 梅林 也 有 一家 店 ， ...,"[142, 113, 1, 28, 1, 56, 914, 77, 70, 1, 29, 1...",1
